In [2]:
#pip install wikipedia-api pandas requests
import wikipediaapi  # API pour accéder à Wikipedia
import pandas as pd  # Pour structurer les données
import requests  # Requêtes HTTP à l'API

# Initialisation de l'API Wikipedia avec un User-Agent personnalisé
user_agent = "GenderAnalysisProject/1.0 (contact@example.com)"  # Remplacez par votre email ou URL de contact
wiki = wikipediaapi.Wikipedia(language='en', user_agent=user_agent)  # Langue: anglais


# Fonction pour explorer une catégorie et ses sous-catégories pour obtenir les pages d'utilisateurs
def get_wikipedians_by_category(category_name):
    """
    Récupère une liste de contributeurs Wikipedia à partir d'une catégorie donnée, en parcourant les sous-catégories.
    :param category_name: Nom de la catégorie (par exemple, 'Female Wikipedians')
    :return: Liste des contributeurs
    """
    category = wiki.page(f"Category:{category_name}")
    wikipedians = []
    
    if category.exists():
        def process_category(cat):
            for member in cat.categorymembers.values():
                # Si la page est un utilisateur, on l'ajoute
                if member.ns == wikipediaapi.Namespace.USER:
                    wikipedians.append(member.title)
                # Si c'est une sous-catégorie, on la traite récursivement
                elif member.ns == wikipediaapi.Namespace.CATEGORY:
                    process_category(member)
                    
        process_category(category)
    else:
        print(f"La catégorie {category_name} n'existe pas.")
    
    return wikipedians

# Collecter les contributeurs des trois catégories
female_wikipedians = get_wikipedians_by_category("Female_Wikipedians")
male_wikipedians = get_wikipedians_by_category("Male_Wikipedians")
transgender_wikipedians = get_wikipedians_by_category("Transgender_Wikipedians")

# Affichage des résultats (10 premiers contributeurs pour chaque catégorie)
print(f"Contributeurs (Femmes): {female_wikipedians[:10]}")
print(f"Contributeurs (Hommes): {male_wikipedians[:10]}")
print(f"Contributeurs (Transgenres): {transgender_wikipedians[:10]}")

# Enregistrer les résultats dans des fichiers CSV
pd.DataFrame(female_wikipedians, columns=['Name']).to_csv("female_wikipedians.csv", index=False)
pd.DataFrame(male_wikipedians, columns=['Name']).to_csv("male_wikipedians.csv", index=False)
pd.DataFrame(transgender_wikipedians, columns=['Name']).to_csv("transgender_wikipedians.csv", index=False)


KeyboardInterrupt: 

In [4]:
import wikipediaapi  # API pour accéder à Wikipedia
import pandas as pd  # Pour structurer les données
import requests
import time
import re
import urllib.parse

# Initialisation de l'API Wikipedia avec un User-Agent personnalisé
user_agent = "GenderAnalysisProject/1.0 (contact@example.com)"  # Remplacez par votre email ou URL de contact
wiki = wikipediaapi.Wikipedia(language='en', user_agent=user_agent)  # Langue: anglais

# Liste des langues à explorer
languages = ['en', 'fr', 'de', 'es', 'it', 'ru', 'ja', 'zh']

# Fonction pour explorer une catégorie et ses sous-catégories pour obtenir les pages d'utilisateurs
def get_wikipedians_by_category(category_name):
    """
    Récupère une liste de contributeurs Wikipedia à partir d'une catégorie donnée, en parcourant les sous-catégories.
    :param category_name: Nom de la catégorie (par exemple, 'Female Wikipedians')
    :return: Liste des contributeurs
    """
    category = wiki.page(f"Category:{category_name}")
    wikipedians = []

    if category.exists():
        def process_category(cat):
            for member in cat.categorymembers.values():
                # Si la page est un utilisateur, on l'ajoute
                if member.ns == wikipediaapi.Namespace.USER:
                    username = member.title
                    # Supprimer le préfixe "User:" s'il est présent
                    if username.startswith('User:'):
                        username = username[len('User:'):]
                    username = username.strip()  # Supprimer les espaces superflus
                    wikipedians.append(username)
                # Si c'est une sous-catégorie, on la traite récursivement
                elif member.ns == wikipediaapi.Namespace.CATEGORY:
                    process_category(member)

        process_category(category)
    else:
        print(f"La catégorie {category_name} n'existe pas.")

    return wikipedians

# Collecter les contributeurs des trois catégories
female_wikipedians = get_wikipedians_by_category("Female_Wikipedians")
male_wikipedians = get_wikipedians_by_category("Male_Wikipedians")
transgender_wikipedians = get_wikipedians_by_category("Transgender_Wikipedians")

# Fonction pour extraire la localisation depuis le contenu de la page utilisateur
def extract_location_from_content(content):
    patterns = [
    r'\{\{\s*[Uu]ser location\s*\|\s*([^}|]+)',                    # {{User location|Location}}
    r'\{\{\s*[Uu]tilisateur\s*[Ll]ocalité\s*\|\s*([^}|]+)',        # {{Utilisateur Localité|Location}}
    r'This user lives in\s*[:]*\s*(.*?)\.',                        # "This user lives in: Location."
    r'This user lives in the\s*[:]*\s*(.*?)\.',                    # "This user lives in: Location."
    r'This user is from\s*[:]*\s*(.*?)\.',                         # "This user is from: Location."
    r'This user resides in\s*[:]*\s*(.*?)\.',                      # "This user resides in: Location."
    r'This user comes from\s*[:]*\s*(.*?)\.',                      # "This user comes from: Location."
    r'This user is located in\s*[:]*\s*(.*?)\.',                   # "This user is located in: Location."
    r'This user was born in\s*[:]*\s*(.*?)\.',                     # "This user was born in: Location."
    r'This user is living in\s*[:]*\s*(.*?)\.',                    # "This user is living in: Location."
    r'I (?:live|am from|come from|reside in|was born in|am located in)\s*[:]*\s*(.*?)\s*(?:\.|,|\n|$)',  # "I live in: Location"
    r'My location is\s*[:]*\s*(.*?)\s*(?:\.|,|\n|$)',              # "My location is: Location"
    r'Resides in\s*[:]*\s*(.*?)\s*(?:\.|,|\n|$)',                  # "Resides in: Location"
    r'From\s*[:]*\s*(.*?)\s*(?:\.|,|\n|$)',                        # "From: Location"
    r'Based in\s*[:]*\s*(.*?)\s*(?:\.|,|\n|$)',                    # "Based in: Location"
    r'Living in\s*[:]*\s*(.*?)\s*(?:\.|,|\n|$)',                   # "Living in: Location"
    r'Born in\s*[:]*\s*(.*?)\s*(?:\.|,|\n|$)',                     # "Born in: Location"
    r'Home\s*[:=]\s*(.*?)\s*(?:\.|,|\n|$)',                        # "Home: Location" ou "Home = Location"
    r'Location\s*[:=]\s*(.*?)\s*(?:\.|,|\n|$)',                    # "Location: Location" ou "Location = Location"
    r'Country\s*[:=]\s*(.*?)\s*(?:\.|,|\n|$)',                     # "Country: Location" ou "Country = Location"
    r'City\s*[:=]\s*(.*?)\s*(?:\.|,|\n|$)',                        # "City: Location" ou "City = Location"
    r'\{\{\s*[Uu]ser\s*[Gg]eolocation\s*\|\s*(.*?)\s*\}\}',        # {{User Geolocation|Location}}
    r'\{\{\s*[Uu]serbox\s*\|\s*location\s*=\s*(.*?)\s*\}\}',       # {{Userbox|location=Location}}
    r'\{\{\s*[Uu]ser\s*[Ll]ocation\s*\|\s*(.*?)\s*\}\}',           # {{User Location|Location}}
    r'\{\{\s*[Uu]ser\s*[Ff]rom\s*\|\s*(.*?)\s*\}\}',               # {{User From|Location}}
    r'\|\s*country\s*=\s*(.*?)\s*(?:\||\n)',                       # "| country = Location"
    r'\|\s*location\s*=\s*(.*?)\s*(?:\||\n)',                      # "| location = Location"
    r'\|\s*residence\s*=\s*(.*?)\s*(?:\||\n)',                     # "| residence = Location"
    r'\|\s*hometown\s*=\s*(.*?)\s*(?:\||\n)',                      # "| hometown = Location"
    r'==\s*Location\s*==\s*(.*?)\s*(?:==|\n\n)',                   # "== Location == Location information"
    r'\{\{\s*[Bb]abel\s*\|\s*(.*?)\}\}',                           # {{Babel|...|Location}}
    r'\{\{\s*[Uu]ser\s*[Mm]aps\s*\|\s*(.*?)\s*\}\}',               # {{User Maps|Location}}
]


    for pattern in patterns:
        match = re.search(pattern, content, re.IGNORECASE)
        if match:
            location = match.group(1).strip()
            # Nettoyage de la localisation pour enlever les balises ou les caractères spéciaux
            location = re.sub(r'<.*?>', '', location)
            location = re.sub(r'\[\[|\]\]', '', location)
            location = location.split('|')[0].strip()
            return location
    return "Unknown"

# Fonction pour obtenir la localisation de l'utilisateur
def get_user_location(username, lang='en'):
    username_encoded = urllib.parse.quote(username)
    url_user = f"https://{lang}.wikipedia.org/w/api.php"
    params_user = {
        "action": "query",
        "titles": f"User:{username_encoded}",
        "prop": "revisions",
        "rvprop": "content",
        "format": "json",
        "formatversion": "2"
    }

    try:
        response_user = requests.get(url_user, params=params_user)
        if response_user.status_code == 200:
            data_user = response_user.json()
            pages = data_user.get('query', {}).get('pages', [])
            for page in pages:
                if 'revisions' in page:
                    content = page['revisions'][0].get('content', '')
                    location = extract_location_from_content(content)
                    if location != "Unknown":
                        return location
    except Exception as e:
        print(f"Erreur lors de la récupération de la localisation pour {username} en {lang}: {e}")
    return "Unknown"

# Fonction pour obtenir les contributions d'un utilisateur
def get_user_contributions(username, limit=20):
    contributions = []
    username_encoded = urllib.parse.quote(username)
    for lang in languages:
        url_contrib = f"https://{lang}.wikipedia.org/w/api.php"
        params_contrib = {
            "action": "query",
            "list": "usercontribs",
            "ucuser": username_encoded,
            "uclimit": limit,
            "format": "json"
        }

        try:
            response_contrib = requests.get(url_contrib, params=params_contrib)
            if response_contrib.status_code == 200:
                data_contrib = response_contrib.json()
                if 'query' in data_contrib and 'usercontribs' in data_contrib['query']:
                    for contrib in data_contrib['query']['usercontribs']:
                        contributions.append({
                            "title": contrib['title'],
                            "timestamp": contrib['timestamp'],
                            "language": lang
                        })
            time.sleep(0.5)  # Pause pour éviter de surcharger les serveurs
        except Exception as e:
            print(f"Erreur lors de la récupération des contributions pour {username} en {lang}: {e}")
    return contributions

# Fonction pour collecter les contributions et la localisation par genre
def collect_contributions_and_location(wikipedians, gender):
    for user in wikipedians:
        username = user.strip()
        # Ignorer les utilisateurs suspects
        if '/' in username or len(username) == 0:
            print(f"Ignorer l'utilisateur suspect: {username}")
            continue
        print(f"Traitement de l'utilisateur: {username}")

        # Obtenir la localisation (par défaut en anglais)
        region = get_user_location(username, lang='en')
        print(f"Localisation trouvée: {region}")

        # Obtenir les contributions de l'utilisateur
        contributions = get_user_contributions(username)
        print(f"Nombre de contributions trouvées: {len(contributions)}")

        for contrib in contributions:
            all_contributions.append({
                "User": username,
                "Title": contrib['title'],
                "Timestamp": contrib['timestamp'],
                "Gender": gender,
                "Language": contrib.get('language', 'unknown'),
                "Region": region if region else 'unknown'
            })
        # Délai pour éviter de surcharger les serveurs de Wikipédia
        time.sleep(1)

# Liste pour stocker toutes les contributions avec la localisation
all_contributions = []

# Collecter les contributions et la localisation pour chaque genre
collect_contributions_and_location(female_wikipedians[:300], 'Female')
collect_contributions_and_location(male_wikipedians[:300], 'Male')
collect_contributions_and_location(transgender_wikipedians[:300], 'Transgender')

# Créer un DataFrame avec toutes les contributions
df_contributions = pd.DataFrame(all_contributions)

# Sauvegarder les données dans un fichier CSV
df_contributions.to_csv("wikipedia_contributions_with_geography5.csv", index=False)

# Afficher les premières lignes du DataFrame
print(df_contributions.head())


Traitement de l'utilisateur: !hauntaholics
Localisation trouvée: Unknown
Nombre de contributions trouvées: 0
Traitement de l'utilisateur: .huepow
Localisation trouvée: United States
Nombre de contributions trouvées: 20
Traitement de l'utilisateur: .pixiedzt
Localisation trouvée: Unknown
Nombre de contributions trouvées: 1
Traitement de l'utilisateur: *Mikaplayz17*
Localisation trouvée: Unknown
Nombre de contributions trouvées: 0
Traitement de l'utilisateur: *Yukio Okumura*
Localisation trouvée: Unknown
Nombre de contributions trouvées: 0
Traitement de l'utilisateur: $ugar$weet
Localisation trouvée: Unknown
Nombre de contributions trouvées: 0
Traitement de l'utilisateur: 0xF8E8
Localisation trouvée: Providence
Nombre de contributions trouvées: 48
Traitement de l'utilisateur: 1Krystal.Stevens
Localisation trouvée: Winchester
Nombre de contributions trouvées: 14
Traitement de l'utilisateur: 2Cute4U
Localisation trouvée: Unknown
Nombre de contributions trouvées: 20
Traitement de l'utilisat